In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src import find_duplicates

ImportError: cannot import name 'find_duplicates'

# Load data

In [ ]:
filename = 'data/raw/targeting_model_data.csv' 
data = pd.read_csv(filename)

In [ ]:
# according to panda Columns 49, 191, 476 have mixed data types.

In [ ]:
data.info()

In [ ]:
len(data.columns)

In [ ]:
data.head()

In [ ]:
data.iloc[:,2].unique()

In [ ]:
data.iloc[:,101:150]

# Clean up columns by finding duplicates 

In [ ]:
from pandas.core.common import array_equivalent


def find_duplicates(df):
    """Finds the names of duplicate columns. 

    Finds the different data types in a dataframe. 
    Creates a dictionary file with different data types as keys and the pertinent columns as keys. 
    
    
    Parameters
    ----------
    df : data : pandas.DataFrame
        dataframe to search for duplicates 


    Returns
    -------
    duplicates : list
        The list of duplicate column names.  
    """
    # emptly list to save duplicates
    duplicates = []

    # group the different data types into a dictionary
    data_groups = df.columns.to_series().groupby(df.dtypes).groups

    for group, columns in data_groups.items():

        sliced_columns_names = df[columns].columns
        sliced_df = df[columns]
        len_columns = len(sliced_columns_names)

        for position1 in range(len_columns):
            column1 = sliced_df.iloc[:, position1].values
            for position2 in range(position1 + 1, len_columns):
                column2 = sliced_df.iloc[:, position2].values
                if array_equivalent(column1, column2):
                    duplicates.append(sliced_columns_names[position1])
                    break

    return duplicates

In [ ]:
#%%timeit
dups = find_duplicates(data)

In [ ]:
dups

In [ ]:
dups2

In [ ]:
np.array_equal(data.loc[:,'ibe1274BHMXG3244U'], data.loc[:,'ibe1270UUGLZ8167Q'])

In [ ]:
data.loc[:,'FLOZVPMFT4626A':]

In [ ]:
df.columns

In [ ]:
df.drop(columns=duplicates)

In [ ]:
df

In [ ]:
print(data['ibe1270UUGLZ8167Q'].equals(data['ibe1271UEOXF6805V']))

In [ ]:
df.columns.to_series().groupby(df.dtypes).groups

In [ ]:
%%timeit
duplicate_columns(df)

In [ ]:
%%timeit
array_equivalent(data['FLOZVPMFT4626A'],data[ 'ibe1270YRFHJ3350O'] )

In [ ]:
%%timeit
data['FLOZVPMFT4626A'].equals(data[ 'ibe1270YRFHJ3350O'])

In [ ]:
%%timeit
np.array_equal(data['FLOZVPMFT4626A'],data[ 'ibe1270YRFHJ3350O'] )

In [ ]:
%%timeit
np.array_equal(data['FLOZVPMFT4626A'].values,data[ 'ibe1270YRFHJ3350O'].values )